<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_56.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!gdown https://drive.google.com/uc?id=1ytJdl1n4e2Y-F8a8mJab0S_4TgyrkE-4

Downloading...
From: https://drive.google.com/uc?id=1ytJdl1n4e2Y-F8a8mJab0S_4TgyrkE-4
To: /content/joint_representations.zip
100% 1.39G/1.39G [00:30<00:00, 45.5MB/s]


In [ ]:
!unzip /content/joint_representations.zip

In [1]:
!gdown https://drive.google.com/uc?id=1wgl3QGXZ4m2aLg3T-1TDXQqSP31RuXgL

Downloading...
From: https://drive.google.com/uc?id=1wgl3QGXZ4m2aLg3T-1TDXQqSP31RuXgL
To: /content/hateful_train+test_unseen.zip
100% 3.27G/3.27G [01:02<00:00, 52.4MB/s]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Validation


In [ ]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [32]:
model = ViTForImageClassification.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000")

loading configuration file /content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "0",
    "1": "1"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": "0",
    "1": "1"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "problem_type": "single_label_classification",
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.33.1"
}

loading weights file /content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000/pytorch_model.bin
All model checkpoint weights were used when ini

# Standard Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/mulitmodal/test", split="train")

In [ ]:
ds

# 100% Missing Modality Experiment

In [ ]:
column_names = ds.column_names

print(column_names)

In [ ]:
import os
def filename_ends_with_4(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [ ]:
ds = ds.filter(filename_ends_with_4)

In [ ]:
ds

In [ ]:
for i in range(150,500):
    sample = ds[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

# Partial Missing Modality Experiment

In [5]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [6]:
ds_f = load_dataset("imagefolder", data_dir="/content/hateful_ViT1/test", split="train")

Resolving data files:   0%|          | 0/2000 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
ds_f

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [8]:
length_to_keep = int(len(ds_f) * 0.7)
print(length_to_keep)

1400


In [9]:
fused_ds = ds_f.select(indices=range(length_to_keep))

In [10]:
fused_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 1400
})

In [13]:
ds_joint = load_dataset("imagefolder", data_dir="/content/mulitmodal/test", split="train")

Resolving data files:   0%|          | 0/4000 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
import os
def filename_ends_with_4(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [15]:
ds_imgs = ds_joint.filter(filename_ends_with_4)

Filter:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [16]:
ds_imgs

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [ ]:
for i in range(10,100):
    sample = ds_imgs[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

In [20]:
length = len(ds_imgs)
length_to_keep = int(length * 0.3)
start_index = length - length_to_keep
missing_ds = ds_imgs.select(indices=range(start_index, length))

In [21]:
missing_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 600
})

In [22]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [23]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

# New Section

In [24]:
labels = ds.features["label"]
labels

ClassLabel(names=['0', '1'], id=None)

In [25]:
labels.int2str(ds[532]["label"])

'0'

In [26]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [27]:
dataset = ds.with_transform(transform)

In [28]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [33]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [30]:
labels = ds.features["label"].names
labels

['0', '1']

In [31]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [34]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [35]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [36]:
batch_size = 32
optimizer = AdamW(model.parameters(), lr=1e-5)

In [37]:
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [38]:
model = model.to(device)
model.eval()
predictions, labels = [], []
valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    loss = outputs.loss
    valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
AUROC_score = roc_auc_score(labels, predictions)

In [40]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("AUROC_score: ", AUROC_score)

Accuracy:  0.736
precision:  0.7319127605390479
f1_score:  0.7328494661176636
AUROC_score:  0.7082666666666666


In [41]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[1024  226]
 [ 302  448]]
